# Batch normalization

make possible to train DNNs quite reliably 
You want that the hidden state pre-activation  to be roughly gaussian -> the tanh won't be saturated
So take the hidden state and normalize it to be Gaussian
WE ARE USING IT TO CONTROL THE ACTIVATION IN THE NN 

We want this to be Gaussian only at initialization and not always . We want the NN to move this around to pentially make it  more diffuse, more sharp . So they added the scale and shift

There is coupling between the examples in the batch. But it plays a role of a regularizer

the momentum parameter is for the running mean and the running std 
The affine parameter is form the shift and scale

In [1]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

zsh:1: command not found: wget


In [ ]:
import torch

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))

mean = hpreact.mean(0, keepdim=True) # for example [32, 200] -> [1,200]  we get the mean over all the batches example 
std = hpreact.std(0, keepdim=True)
hpreact = bngain * (hpreact - mean) / std  +  bnbias 

In [ ]:
# calibrate the batch norm layer at the end of the training

with torch.no_grad():
    # pass the training set through
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    # measure the mean and std over the entire training set
    bnmean = hpreact.mean(0, keepdim=True)
    bnstd = hpreact.std(0, keepdim=True)
    
    # So at test time we are fixing the mean and std of the batch norm layer§

In [ ]:
# Let's train a deep neural network

class Linear:
    
    def __init__(self, fan_in, fan_out, bias=True) -> None:
        self.weight = torch.randn((fan_out, fan_in), generator=g) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
        
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] +([] if self.bias is None else [self.bias])
    
class BatchNorm1d:
    
    def __init__(self, dim, momentum=0.1, eps=1e-5) -> None:
        self.momentum = momentum
        self.eps = eps
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a 'momentum update')
        # self.register_buffer('running_mean', torch.zeros(dim))
        # self.register_buffer('running_var', torch.ones(dim))
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
        
    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        x_hat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * x_hat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad(): # exponential moving average
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    

class Tanh:
    
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []
    

n_embd = 10 # the dimensionality of the character vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd),            generator=g)
layers = [
    Linear(n_embd * block_size, n_hidden),  Tanh(),
    Linear(n_hidden, n_hidden),             Tanh(),
    Linear(n_hidden, n_hidden),             Tanh(),
    Linear(n_hidden, n_hidden),             Tanh(),
    Linear(n_hidden, n_hidden),             Tanh(),
    Linear(n_hidden, vocab_size)
]

with torch.no_grad():
    # last layer : make less confident
    layers[-1].weight *= 0.1
    # all other layers : apply gain
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 5/3
            
parameters = [C] + [param for layer in layers for param in layer.parameters()]
print(sum(param.numel() for param in parameters)) # number of parameters in  total
for p in parameters:
    p.requires_grad = True